# Introduction

Ce document résume les jeu de données, méthodologie, et statistiques utilisées pour l'estimation de la souffrance contenue dans les boîtes d'oeufs.

Nous commençons par l'import de la base de données complète d'open food facts obtenue le 31 mars 2025.

De cette base de données, nous ne retenons que les colonnes (goodcol) nécessaires au calcul du poids de souffrance, telles que définies dans le code.




In [ ]:
import sys
sys.path.append("backend")

import duckdb
import requests

import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 1000)


In [ ]:
LOCAL_PARQUET = r"..\data\food.parquet"
SOURCE_PARQUET = 'https://huggingface.co/datasets/openfoodfacts/product-database/resolve/main/food.parquet'

download = input('Télécharger le dernier parquet (4 GO) ? o/n')

if download == 'o':

    with requests.get(SOURCE_PARQUET, stream=True) as r:
        r.raise_for_status()
        with open(LOCAL_PARQUET, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)



In [ ]:


minicol=['code', # ID
 'categories_tags', # déjà présent
 'labels_tags', # déjà présent
 'product_name', # déjà présent
 'generic_name',
 'quantity',
 'product_quantity_unit',
 'product_quantity',
 'allergens_tags',
 'ingredients_tags',
 'ingredients',
 'countries_tags',
 'images',
 ]

duckdb.execute(f"CREATE OR REPLACE VIEW db_col AS SELECT {','.join(minicol)} FROM '{LOCAL_PARQUET}'")
duckdb.execute("SUMMARIZE db_col").df()

# Identification des oeufs

## Approche additive

Le résumé de la base de données indique qu'il y a 3,78 millions de produits.

On note au passage que `quantity` et `product_quantity`, qui constituent la base de notre approche, nécessiteront un gros travail se nettoyage.

L'identification des boîtes d'oeufs semble assez pédestre, puisque selon l'approche taxonomique il suffirait de sélectionner les produits ayant `en:chicken-eggs` dans `categories_tags`.

Combien y en a-t-il ?


In [ ]:
duckdb.execute("SELECT COUNT(*) FROM db_col WHERE 'en:chicken-eggs' IN categories_tags").df()


## Approche soustractive

On suppose que la taxonomie est incomplète / imparfaite et on tente une approche par soustraction:
plutôt que de prendre les éléments d'oeufs de poule, on sélectionne les oeufs dont on enlève tout ce qui est identifié comme oeufs d'autre animaux, en supposant que le défaut est oeuf de poule.

Afin d'arbitrer entre les deux approches, on compare le nombre d'éléments de cette approche avec le nombre d'éléments précédents et échantillonne quelques éléments afin de voir si cela a du sens.


In [ ]:
ltags=duckdb.execute("SELECT list_distinct(categories_tags) FROM db_col WHERE 'en:eggs' IN categories_tags").df()
set(x for xs in ltags.iloc[:, 0] for x in xs if x.startswith("en") and x.endswith("-eggs"))


Avec cette approche nous ne sommes pas parvenus à retrouver les éléments "ostrich eggs", "guineafowl eggs", etc.
Nous parvenons à cet ensemble d'éléments à exclure :

In [ ]:
pas_poule={'en:chocolate-eggs',
 'en:duck-eggs',
 'en:easter-eggs',
 'en:fish-eggs',
 'en:free-range-duck-eggs',
 'en:quail-eggs',
 'en:raw-quail-eggs',
 'en:savoury-eggs',
 'en:scotch-eggs',
 'en:streamed-eggs',
'en:meals',
'en:snacks',        
'en:meats-and-their-products',
'en:breads'
          }


joint="' NOT IN categories_tags AND '".join(list(pas_poule))
request="CREATE OR REPLACE VIEW eggs AS SELECT * FROM db_col WHERE 'en:eggs' IN categories_tags AND '" +\
         joint+"' NOT IN categories_tags"
#print(request)

duckdb.execute(request)
eggs_from_parquet_duckdb=duckdb.execute("FROM eggs").df()

In [ ]:
eggs_from_parquet_duckdb.sample(50, random_state=10)

Conversion de la synthaxe duckdb en json

In [ ]:
import json
import numpy as np

cols_to_json = []

for col in eggs_from_parquet_duckdb.columns:
    sample = eggs_from_parquet_duckdb[col].dropna().head(20)
    if sample.apply(lambda x: isinstance(x, (list, dict, np.ndarray))).any():
        cols_to_json.append(col)

cols_to_json

cols_to_json_for_import = cols_to_json + ['ingredients']

In [ ]:
eggs_from_parquet = eggs_from_parquet_duckdb.copy()

def ndarray_to_json(arr):
    if isinstance(arr, (list, dict)):
        return json.dumps(arr)
    elif isinstance(arr, np.ndarray):
        return json.dumps(arr.tolist())
    else:
        return arr  # valeur non traitée

for col in cols_to_json_for_import:
    eggs_from_parquet[col] = eggs_from_parquet_duckdb[col].apply(ndarray_to_json)

In [ ]:
with open("../data/cols_to_json.txt", "w") as f:
    json.dump(cols_to_json_for_import, f)

eggs_from_parquet.to_csv("../data/eggs_from_parquet.csv", index=False)
eggs_from_parquet